**Asistente de Generación de Presentaciones en PPT**

Este programa busca acelerar el proceso de generación de presentaciones completas en PowerPoint, apoyando a los expositores con temas como:<br>
<ul>
    <li>Optimización de recursos</li>
    <li>Tiempo destinado a investigación</li>
    <li>Bloqueos creativos</li>
</ul>

**1.** Llamamos a la API de OpenAI desde el archivo config.py

In [ ]:
from config import openai_api_key

import openai
import os

**2.** Definimos la API como una variable environment

In [ ]:
os.environ["OPENAI_API_KEY"] = openai_api_key

**3.** Dividimos el proceso en los siguientes pasos:<br>
<ul>
    <li>Definimos nuestras variables iniciales, como el tema, el número de diapositivas y el nombre de usuario.</li>
    <li>Declaramos dos funciones para poder utilizarlas luegos en un loop, ambas orientadas a establecer el número de diapositivas a codificar, utilizando una validación para garantizar que sea un número entero en un rango determinado. En este caso, permitimos entre 1 y 8 diapositiva.</li>
    <li>Definimos nuestro loop con While True donde:</li>
    <ul>
        <li>Se verifica si tanto el valor inicial del tema como del número de diapositivas son vacíos o igual a 0</li>
        <li>Si cumplen esta característica, se le da la instrucción a la API de que genere los puntos principales para la presentación</li>
        <li>Se le pide al usuario su conformidad respecto al tema y los subtemas</li>
        <li>Si el usuario acepta, se procede a generar el código VBA para copiar y pegar en PowerPoint a través de la opción "Programador"; caso contrario, se le solicita al usuario confirmar si quiere cambiar el tema, el número de diapositivas o ambos parámetros.</li>
        <li>Finalmente se le da al usuario la lista de prompts para que pueda generar las imágenes complementarias en el generados de su prefe</li>
    </ul>
</ul>

In [ ]:
tema_presentacion = ""
numero_diapositivas = 0
nombre_usuario = input("¡Hola! ¿Cuál es tu nombre?")
min_diapos = 1
max_diapos = 8

def solicitar_numero_diapositivas():
    while True:
        try:
            numero = int(input(f"¿Cuántas diapositivas necesitas? Para esta prueba puedes pedirme entre {min_diapos} y {max_diapos} diapositivas."))
            if 1 <= numero <= 8:
                return numero
            else:
                print("Por favor, ingresa un número entre 1 y 8.")
        except ValueError:
            print("No puedo interpretar esa respuesta. Intenta de nuevo.")

def solicitar_nuevo_numero_diapositivas():
    while True:
        try:
            numero = int(input(f"¿Cuántas diapositivas necesitas? Recuerda que puedes pedirme entre {min_diapos} y {max_diapos} diapositivas."))
            if 1 <= numero <= 8:
                return numero
            else:
                print("Por favor, ingresa un número entre 1 y 8.")
        except ValueError:
            print("No puedo interpretar esa respuesta. Intenta de nuevo.")

while True:
    if not tema_presentacion:
        tema_presentacion = input(f"Mucho gusto, {nombre_usuario}. Soy Rony Starp, tu asistente de generación de diapositivas en Powerpoint.\n" +
                                  "¡Vamos a armar tu presentación!\n" +
                                  "¿Con qué tema quieres que te apoye hoy? ")
    if not numero_diapositivas:
        numero_diapositivas = solicitar_numero_diapositivas()

    conversation=[
        {"role":"system","content":"Actúa como Rony Starp, un asistente diseñador experto en la generación de diapositivas en Powerpoint,"\
        +"con más de 10 años de experiencia trabajando para uno de los mejores expositores y oradores a nivel global."\
        +"Ayúdame diseñando el texto de la vista esquema para una presentación en PowerPoint, el texto para cada diapositiva debe"\
        +f"tener título y puntos clave. Enfócate en armar una presentación sobre {tema_presentacion} con {numero_diapositivas} diapositivas,"\
        +"sin hacerme preguntas sobre diseño, ya que no es mi especialidad."},
        {"role":"user","content":f"Dame una lista de {numero_diapositivas} items con ideas centrales que formarán el índice de la"\
        +f"presentación sobre {tema_presentacion}."}
    ]

    response_presentacion=openai.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=conversation,
        max_tokens=1000,
        temperature=0.7
    )

    lista_ideas=response_presentacion.choices[0].message.content

    print(lista_ideas+"\n")

    confirmacion_estructura = input("¿Estás de acuerdo con esta estructura? Responde Sí o No:").lower()
    
    if confirmacion_estructura == "sí" or confirmacion_estructura == "si":
        
        print("¡Genial! Continuaremos con esta estructura.\nDame un momento mientras preparo el siguiente paso...")
        
        conversation=[
            {"role":"system","content":"Eres un asistente útil, especialista en programación de presentación en PowerPoint con VBA."},
            {"role":"user","content":"Crea un código VBA que pueda copiar y pegar en PowerPoint Visual Basic. Dicho código debe permitirme generar "\
            +f"una presentación de {numero_diapositivas} diapositivas sobre el tema {tema_presentacion}, incluyendo los puntos clave "\
            +f"presentes en {lista_ideas}. Todas y cada una de las diapositivas deben incluir al menos un punto clave. Adicionalmente, necesito "\
            +f"que incluyas notas al pie de página con información adicional complementaria referente a {tema_presentacion} que no se haya "\
            +"mencionado ya en los puntos clave. Está totalmente prohibido que las notas no aporten valor. "\
            +"Las notas deben ser únicas para diapositiva. Está prohibido también que uses una función loop del estilo "\
            +"'For i = 1 To pptPresentation.Slides.Count'. Las notas de página deben ser útiles e ir de la mano con los puntos clave. "\
            +f"Comienza tu respuesta con este esquema: 'Aquí tienes el código VBA, {nombre_usuario}' y procede inmediatamente a mostrar el código"}
        ]

        response_vba=openai.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=conversation,
            max_tokens=4000,
            temperature=0.9
        )

        codigo_vba=response_vba.choices[0].message.content

        print(f"{codigo_vba}\n\nDame un momento mientras preparo el último paso...\n")

        conversation=[
            {"role":"system","content":"Eres un asistente útil, especializado en generación de imágenes en base a prompts."},
            {"role":"user","content":f"Crea un total de {numero_diapositivas} prompts para generar imágenes relacionadas a las "\
            +f"{numero_diapositivas} diapositivas generadas anteriormente respecto al tema {tema_presentacion}. Considera los "\
            +f"puntos clave presentes en {lista_ideas}. Presenta los prompts en una lista con bullet points. Cada prompt debe generar "\
            +"una imagen detallada con calidad fotorealista. Tu respuesta debe comenzar con el primer bullet point, sin ninguna palabra "\
            "adicioanl"}
        ]

        response_prompts=openai.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=conversation,
            max_tokens=1000,
            temperature=0.5
        )

        listado_prompts=response_prompts.choices[0].message.content

        print(listado_prompts)

        break
        
    else:
        sugerencias_cambio = input("Comprendo. ¿Te gustaría cambiar el tema, la cantidad de diapositivas o ambos?\n"
                                  +"• Escribe T para cambiar solo el tema.\n"\
                                  +"• Escribe D para cambiar solo el número de diapositivas.\n"\
                                  +"• Escribe A para cambiar ambos.").lower()
        if sugerencias_cambio == "t":
            tema_presentacion = input("Ingresa el nuevo tema para la presentación: ")
        elif sugerencias_cambio == "d":
            numero_diapositivas = solicitar_nuevo_numero_diapositivas()
        elif sugerencias_cambio == "a":
            tema_presentacion = input("Ingresa el nuevo tema para la presentación: ")
            numero_diapositivas = solicitar_nuevo_numero_diapositivas()
        else:
            print("Esa no es una opción válida. Inténtalo de nuevo.")
